# Setup

In [2]:
from sklearn import datasets
import pandas as pd
import numpy as np

iris = datasets.load_iris()
iris_frame = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_frame['target'] = iris.target

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


class SimplePipeline:
    def __init__(self):
        self.frame = None
        self.X_train, self.X_test, self.y_train, self.Y_test = None, None, None, None
        self.model = None
        self.load_dataset()
    
    def load_dataset(self):
        dataset = datasets.load_iris()
        
        # remove units ' (cm)' from variable names
        self.feature_names = [fn[:-5] for fn in dataset.feature_names]
        self.frame = pd.DataFrame(dataset.data, columns=self.feature_names)
        self.frame['target'] = dataset.target

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.frame[self.feature_names], self.frame.target, test_size=0.65, random_state=42)
        
    def train(self, algorithm=LogisticRegression):
        self.model = algorithm(solver='lbfgs', multi_class='auto')
        self.model.fit(self.X_train, self.y_train)
        
    def predict(self, input_data):
        return self.model.predict(input_data)
        
    def get_accuracy(self):
        return self.model.score(X=self.X_test, y=self.y_test)
    
    def run_pipeline(self):
        self.load_dataset()
        self.train()

In [4]:
from sklearn.preprocessing import StandardScaler


class PipelineWithDataEngineering(SimplePipeline):
    def __init__(self):
        super().__init__()
        
        self.scaler = StandardScaler()
        self.scaler.fit(self.X_train)
    
    def apply_scaler(self):
        self.X_train = self.scaler.transform(self.X_train)
        self.X_test = self.scaler.transform(self.X_test)
        
    def predict(self, input_data):
        scaled_input_data = self.scaler.transform(input_data)
        return self.model.predict(scaled_input_data)
                  
    def run_pipeline(self):
        self.load_dataset()
        self.apply_scaler()
        self.train()

# Test
- 1. simple한 benchmark model보다 좋은 성능이 나오는가?
- 2. 이전 모델이 있다고 할 때, 이전 모델보다 좋은 성능이 나오는가?

In [5]:
import unittest
from sklearn.metrics import mean_absolute_error, accuracy_score

In [10]:
class TestIrisPredictions(unittest.TestCase):
    def setUp(self):
        self.pipeline_v1 = SimplePipeline()
        self.pipeline_v2 = PipelineWithDataEngineering()
        self.pipeline_v1.run_pipeline()
        self.pipeline_v2.run_pipeline()

    def test_acc_higher_than_simple_benchmark(self):
        benchmark_predictions = [1.0] * len(self.pipeline_v1.y_test)
        benchmark_acc = accuracy_score(self.pipeline_v1.y_test, benchmark_predictions)
        predictions = self.pipeline_v1.predict(self.pipeline_v1.X_test)
        pred_acc = accuracy_score(self.pipeline_v1.y_test, predictions)

        print(f'model acc: {pred_acc:.3f}, benchmark acc: {benchmark_acc:.3f}')
        self.assertTrue(pred_acc > benchmark_acc)

    def test_acc_compared_to_previous_version(self):
        v1_acc = self.pipeline_v1.get_accuracy()
        v2_acc = self.pipeline_v2.get_accuracy()

        print(f'v1 acc: {v1_acc:.3f}, v2_acc: {v2_acc:.3f}')
        self.assertTrue(v2_acc >= v1_acc)

In [11]:
import sys

suite = unittest.TestLoader().loadTestsFromTestCase(TestIrisPredictions)
unittest.TextTestRunner(verbosity=1, stream=sys.stderr).run(suite)

F.
FAIL: test_acc_compared_to_previous_version (__main__.TestIrisPredictions)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_11315/1281880919.py", line 22, in test_acc_compared_to_previous_version
    self.assertTrue(v2_acc >= v1_acc)
AssertionError: False is not true

----------------------------------------------------------------------
Ran 2 tests in 0.084s

FAILED (failures=1)


v1 acc: 0.969, v2_acc: 0.959
model acc: 0.969, benchmark acc: 0.327


<unittest.runner.TextTestResult run=2 errors=0 failures=1>